In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [53]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
from fastai.tabular.all import *

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
#hide
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [54]:
df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")

In [55]:
df.head().T

In [56]:
df.shape

In [57]:
[print(f"{col:25}\t{df[col].dtype}\t{df[col].nunique()}") for col in df.columns];

In [58]:
# Data Cleaning/Feature engineering

In [59]:
# Passengerid
df[['Group', 'GroupNo']] = df['PassengerId'].str.split('_', 1, expand=True)
df[['Group', 'GroupNo', 'PassengerId']]

In [60]:
# HomePlanet
print(df['HomePlanet'].unique())
print(df['HomePlanet'].value_counts())

In [61]:
# Cabin
df[['deck', 'num', 'side']] = df['Cabin'].str.split('/', expand=True)
df[['deck', 'num', 'side', 'Cabin']]

In [62]:
# side
print(df['side'].unique())
print(df['side'].value_counts())

In [63]:
# Name
df[['FirstName','LastName']] = df['Name'].str.split(' ', expand=True)
df[['FirstName','LastName', 'Name']]

In [64]:
dep_var = 'Transported'

In [65]:
cont,cat = cont_cat_split(df, 5, dep_var=dep_var)

In [66]:
procs = [Categorify, FillMissing, Normalize]

In [67]:
splits = RandomSplitter(valid_pct=0.3, seed=0)(range_of(df))

In [68]:
to = TabularPandas(df, procs, cat, cont, dep_var, splits=splits)

In [69]:
to.all_cols.dtypes

In [70]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y
len(to.train),len(to.valid)

In [71]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [72]:
xgr = XGBClassifier()
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(xgr, xs, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [75]:
xgr = XGBClassifier().fit(xs, y)
m_acc(xgr, xs, y), m_acc(xgr, valid_xs, valid_y) 

In [74]:
def m_acc(m, xs, y): return sklearn.metrics.accuracy_score(y, m.predict(xs))

In [84]:
df_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
df_test[['Group', 'GroupNo']] = df_test['PassengerId'].str.split('_', 1, expand=True)
df_test[['deck', 'num', 'side']] = df_test['Cabin'].str.split('/', expand=True)
df_test[['FirstName','LastName']] = df_test['Name'].str.split(' ', expand=True)
cont_test,cat_test = cont_cat_split(df_test, 5, dep_var=None)
procs_test = [Categorify, FillMissing, Normalize]
to_test = TabularPandas(df_test, procs_test, cat_test, cont_test, y_names=None)
xs_test = to_test.train.xs
preds = xgr.predict(xs_test)
my_submission = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Transported': preds.astype(bool)})
my_submission.to_csv('submission_xgr.csv', index=False)